In [1]:
# Dependencies
import pandas as pd
import requests
import json
import datetime
import numpy as np

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy_utils import database_exists, create_database

from ps_wd import pswd

In [2]:
# Run a request to endpoint and convert result to json
# url = "https://data.sfgov.org/resource/g8m3-pdis.json"
# sf_data = requests.get(url).json()

# Print the json
#print(sf_data)

# Print the json (pretty printed)
# print(json.dumps(sf_data, indent=4, sort_keys=True))

In [3]:
#Read the csv file 
sf_csv = "Registered_Business_Locations_-_San_Francisco.csv"
sf_df = pd.read_csv(sf_csv)
sf_df.head()

C:\Users\User\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Location Id,Business Account Number,Ownership Name,DBA Name,Street Address,City,State,Source Zipcode,Business Start Date,Business End Date,...,Supervisor District,Neighborhoods - Analysis Boundaries,Business Corridor,Business Location,UniqueID,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Neighborhoods
0,0031446-01-001,31446,Yong Kee Co Inc,Yong Kee Co Inc,732 Jackson St,San Francisco,CA,94133,05/31/2013,NaN,...,3.0,Chinatown,NaN,POINT (-122.40721 37.795982),0031446-01-001-0031446-5300-5399-05-31-2013,104.0,6.0,3.0,6.0,104.0
1,0051039-02-001,51039,Busuttel Joseph F,3615 19th St Apts,3615 19th St,San Francisco,CA,94110,10/01/1968,NaN,...,8.0,Mission,NaN,POINT (-122.423996 37.75992),0051039-02-001-0051039-5300-5399-04-22-1977,37.0,3.0,5.0,20.0,37.0
2,0078067-01-999,78067,R. S. Hughes Company Inc,Hughes R S Co Inc,7303 Edgewater Dr Oakland Unit B,Oakland,CA,94621,10/01/1968,NaN,...,NaN,NaN,NaN,POINT (-122.20762 37.74483),0078067-01-999-0078067-4200-4299-10-01-1968,NaN,NaN,NaN,NaN,NaN
3,0080538-01-001,80538,Kwang Chun Tsang & So Lan,333 31st Av Apts,333 31st Ave,San Francisco,CA,94121,07/20/1973,NaN,...,1.0,Outer Richmond,NaN,POINT (-122.491425 37.78295),0080538-01-001-0080538-5300-5399-07-20-1973,8.0,8.0,4.0,29.0,8.0
4,0135472-01-001,135472,Saddul Oscar Allado Md Inc,Dr Oscar Allado Saddul-Md,2460 Mission St Ste 215,San Francisco,CA,94110,05/15/1982,NaN,...,9.0,Mission,NaN,POINT (-122.419 37.757675),0135472-01-001-0135472-5400-5499-05-15-1982,53.0,3.0,2.0,20.0,53.0


In [4]:
# drop columns that you recognize as unnecessary 
sf_df.drop(["Location Id","Business Account Number","Ownership Name","Mail Address","Mail State", "Mail City","Mail Zipcode","Parking Tax","Transient Occupancy Tax","LIC Code","LIC Code Description","Supervisor District","Business Corridor","UniqueID","SF Find Neighborhoods","Current Police Districts","Current Supervisor Districts","Analysis Neighborhoods","Neighborhoods"], axis = 1, inplace = True) 

#drop columns that you recognize as unnecessary 
# sf_df.drop(["Location_Id","Business_Account_Number","Ownership_Name","Mail_Address","Mail_State", "Mail_City","Mail_Zipcode","Parking_Tax","Transient_Occupancy_Tax","LIC_Code","LIC_Code_Description","Supervisor_District","Business_Corridor","UniqueID","SF_Find_Neighborhoods","Current_Police_Districts","Current_Supervisor_Districts","Analysis_Neighborhoods","Neighborhoods"], axis = 1, inplace = True) 
  

sf_df.head()

,DBA Name,Street Address,City,State,Source Zipcode,Business Start Date,Business End Date,Location Start Date,Location End Date,NAICS Code,NAICS Code Description,Neighborhoods - Analysis Boundaries,Business Location
0,Yong Kee Co Inc,732 Jackson St,San Francisco,CA,94133,05/31/2013,NaN,05/31/2013,NaN,5300-5399,Real Estate and Rental and Leasing Services,Chinatown,POINT (-122.40721 37.795982)
1,3615 19th St Apts,3615 19th St,San Francisco,CA,94110,10/01/1968,NaN,04/22/1977,NaN,5300-5399,Real Estate and Rental and Leasing Services,Mission,POINT (-122.423996 37.75992)
2,Hughes R S Co Inc,7303 Edgewater Dr Oakland Unit B,Oakland,CA,94621,10/01/1968,NaN,10/01/1968,03/30/2014,4200-4299,Wholesale Trade,NaN,POINT (-122.20762 37.74483)
3,333 31st Av Apts,333 31st Ave,San Francisco,CA,94121,07/20/1973,NaN,07/20/1973,NaN,5300-5399,Real Estate and Rental and Leasing Services,Outer Richmond,POINT (-122.491425 37.78295)
4,Dr Oscar Allado Saddul-Md,2460 Mission St Ste 215,San Francisco,CA,94110,05/15/1982,NaN,05/15/1982,NaN,5400-5499,"Professional, Scientific, and Technical Services",Mission,POINT (-122.419 37.757675)


In [5]:
sf_df['Business Location'].isna().sum()

1

In [6]:
#Cleaning coordinates format

#Remove the string "POINT" from location coordinates
col = sf_df['Business Location'] 
split1 = [str(x).replace('POINT ', '') for x in col]
split1[0:4]

['(-122.40721 37.795982)',
 '(-122.423996 37.75992)',
 '(-122.20762 37.74483)',
 '(-122.491425 37.78295)']

In [7]:
#Remove paranthesis from coordinates
split2 = [x.replace('(', '').replace(')', '') for x in split1]
split2[0:4]

['-122.40721 37.795982',
 '-122.423996 37.75992',
 '-122.20762 37.74483',
 '-122.491425 37.78295']

In [8]:
#Split the string to get lat & lan 
split3 =[x.split(' ') for x in split2]
split3[0:4]

[['-122.40721', '37.795982'],
 ['-122.423996', '37.75992'],
 ['-122.20762', '37.74483'],
 ['-122.491425', '37.78295']]

In [9]:
#Create dataframe from the list of coordinates
df = pd.DataFrame(split3)

#Add column names
df.columns = ['longitude', 'latitude']
df.head()

,longitude,latitude
0,-122.40721,37.795982
1,-122.423996,37.75992
2,-122.20762,37.74483
3,-122.491425,37.78295
4,-122.419,37.757675


In [10]:
df['latitude'].isna().sum()

1

In [11]:
#Merge the lat,lan df with main df
merged_df = pd.concat([sf_df, df], axis=1)
merged_df

,DBA Name,Street Address,City,State,Source Zipcode,Business Start Date,Business End Date,Location Start Date,Location End Date,NAICS Code,NAICS Code Description,Neighborhoods - Analysis Boundaries,Business Location,longitude,latitude
0,Yong Kee Co Inc,732 Jackson St,San Francisco,CA,94133,05/31/2013,NaN,05/31/2013,NaN,5300-5399,Real Estate and Rental and Leasing Services,Chinatown,POINT (-122.40721 37.795982),-122.40721,37.795982
1,3615 19th St Apts,3615 19th St,San Francisco,CA,94110,10/01/1968,NaN,04/22/1977,NaN,5300-5399,Real Estate and Rental and Leasing Services,Mission,POINT (-122.423996 37.75992),-122.423996,37.75992
2,Hughes R S Co Inc,7303 Edgewater Dr Oakland Unit B,Oakland,CA,94621,10/01/1968,NaN,10/01/1968,03/30/2014,4200-4299,Wholesale Trade,NaN,POINT (-122.20762 37.74483),-122.20762,37.74483
3,333 31st Av Apts,333 31st Ave,San Francisco,CA,94121,07/20/1973,NaN,07/20/1973,NaN,5300-5399,Real Estate and Rental and Leasing Services,Outer Richmond,POINT (-122.491425 37.78295),-122.491425,37.78295
4,Dr Oscar Allado Saddul-Md,2460 Mission St Ste 215,San Francisco,CA,94110,05/15/1982,NaN,05/15/1982,NaN,5400-5499,"Professional, Scientific, and Technical Services",Mission,POINT (-122.419 37.757675),-122.419,37.757675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267821,Yulin Li,58 West Portal Ave 568,San Francisco,CA,94127,01/08/2021,NaN,01/08/2021,NaN,5100-5199,Information,West of Twin Peaks,POINT (-122.466515 37.74023),-122.466515,37.74023
267822,Brian Spiers Development Llc,1430 Geysers Rd,Geyserville,CA,95441,05/01/2012,NaN,05/01/2012,NaN,5300-5399,Real Estate and Rental and Leasing Services,NaN,POINT (-122.8413 38.690006),-122.8413,38.690006
267823,Marco Flores,659 Paris St,San Francisco,CA,94112,11/24/2020,NaN,11/24/2020,NaN,2300-2399,Construction,Excelsior,POINT (-122.43744 37.71837),-122.43744,37.71837
267824,"Specialty's 150 Spear, Llc",150 Spear St,San Francisco,CA,94105-1535,11/03/2020,NaN,11/03/2020,NaN,7220-7229,Food Services,Financial District/South Beach,POINT (-122.3933 37.791893),-122.3933,37.791893


In [12]:
merged_df.isna().sum()

DBA Name                                    0
Street Address                              4
City                                      191
State                                     618
Source Zipcode                            203
Business Start Date                         0
Business End Date                      160065
Location Start Date                         0
Location End Date                      131739
NAICS Code                              97447
NAICS Code Description                  97447
Neighborhoods - Analysis Boundaries     47394
Business Location                           1
longitude                                   0
latitude                                    1
dtype: int64

In [13]:
merged_df.dtypes

DBA Name                               object
Street Address                         object
City                                   object
State                                  object
Source Zipcode                         object
Business Start Date                    object
Business End Date                      object
Location Start Date                    object
Location End Date                      object
NAICS Code                             object
NAICS Code Description                 object
Neighborhoods - Analysis Boundaries    object
Business Location                      object
longitude                              object
latitude                               object
dtype: object

In [14]:
merged_df.set_axis(['business_name','address', 'city', 'state', 'zipcode', 'busi_start_dt', 'busi_end_dt', 'loc_start_dt',
               'loc_end_dt', 'NAICS_code', 'busi_type', 'neighborhood', 'coordinates', 'longitude', 'latitude'], axis=1, inplace=True)
merged_df.head()

,business_name,address,city,state,zipcode,busi_start_dt,busi_end_dt,loc_start_dt,loc_end_dt,NAICS_code,busi_type,neighborhood,coordinates,longitude,latitude
0,Yong Kee Co Inc,732 Jackson St,San Francisco,CA,94133,05/31/2013,NaN,05/31/2013,NaN,5300-5399,Real Estate and Rental and Leasing Services,Chinatown,POINT (-122.40721 37.795982),-122.40721,37.795982
1,3615 19th St Apts,3615 19th St,San Francisco,CA,94110,10/01/1968,NaN,04/22/1977,NaN,5300-5399,Real Estate and Rental and Leasing Services,Mission,POINT (-122.423996 37.75992),-122.423996,37.75992
2,Hughes R S Co Inc,7303 Edgewater Dr Oakland Unit B,Oakland,CA,94621,10/01/1968,NaN,10/01/1968,03/30/2014,4200-4299,Wholesale Trade,NaN,POINT (-122.20762 37.74483),-122.20762,37.74483
3,333 31st Av Apts,333 31st Ave,San Francisco,CA,94121,07/20/1973,NaN,07/20/1973,NaN,5300-5399,Real Estate and Rental and Leasing Services,Outer Richmond,POINT (-122.491425 37.78295),-122.491425,37.78295
4,Dr Oscar Allado Saddul-Md,2460 Mission St Ste 215,San Francisco,CA,94110,05/15/1982,NaN,05/15/1982,NaN,5400-5499,"Professional, Scientific, and Technical Services",Mission,POINT (-122.419 37.757675),-122.419,37.757675


In [15]:
#Get details of the data that is not based in San Francisco 
indexNames = merged_df[ merged_df['city'] != ("San Francisco" or "San+Francisco") ].index
# # Delete these row indexes from dataFrame
merged_df.drop(indexNames , inplace=True)
merged_df.head()

,business_name,address,city,state,zipcode,busi_start_dt,busi_end_dt,loc_start_dt,loc_end_dt,NAICS_code,busi_type,neighborhood,coordinates,longitude,latitude
0,Yong Kee Co Inc,732 Jackson St,San Francisco,CA,94133,05/31/2013,NaN,05/31/2013,NaN,5300-5399,Real Estate and Rental and Leasing Services,Chinatown,POINT (-122.40721 37.795982),-122.40721,37.795982
1,3615 19th St Apts,3615 19th St,San Francisco,CA,94110,10/01/1968,NaN,04/22/1977,NaN,5300-5399,Real Estate and Rental and Leasing Services,Mission,POINT (-122.423996 37.75992),-122.423996,37.75992
3,333 31st Av Apts,333 31st Ave,San Francisco,CA,94121,07/20/1973,NaN,07/20/1973,NaN,5300-5399,Real Estate and Rental and Leasing Services,Outer Richmond,POINT (-122.491425 37.78295),-122.491425,37.78295
4,Dr Oscar Allado Saddul-Md,2460 Mission St Ste 215,San Francisco,CA,94110,05/15/1982,NaN,05/15/1982,NaN,5400-5499,"Professional, Scientific, and Technical Services",Mission,POINT (-122.419 37.757675),-122.419,37.757675
6,Bakesale Betty/hot Dog Bill East 600,1 Warriors Way Lvl 600 East,San Francisco,CA,94158,07/01/1987,NaN,05/28/2019,NaN,7220-7229,Food Services,Mission Bay,POINT (-122.386406 37.768967),-122.386406,37.768967


In [16]:
#strip the zipcode to 5 digits only
merged_df['zipcode']= merged_df['zipcode'].str[:5]
merged_df.head()

,business_name,address,city,state,zipcode,busi_start_dt,busi_end_dt,loc_start_dt,loc_end_dt,NAICS_code,busi_type,neighborhood,coordinates,longitude,latitude
0,Yong Kee Co Inc,732 Jackson St,San Francisco,CA,94133,05/31/2013,NaN,05/31/2013,NaN,5300-5399,Real Estate and Rental and Leasing Services,Chinatown,POINT (-122.40721 37.795982),-122.40721,37.795982
1,3615 19th St Apts,3615 19th St,San Francisco,CA,94110,10/01/1968,NaN,04/22/1977,NaN,5300-5399,Real Estate and Rental and Leasing Services,Mission,POINT (-122.423996 37.75992),-122.423996,37.75992
3,333 31st Av Apts,333 31st Ave,San Francisco,CA,94121,07/20/1973,NaN,07/20/1973,NaN,5300-5399,Real Estate and Rental and Leasing Services,Outer Richmond,POINT (-122.491425 37.78295),-122.491425,37.78295
4,Dr Oscar Allado Saddul-Md,2460 Mission St Ste 215,San Francisco,CA,94110,05/15/1982,NaN,05/15/1982,NaN,5400-5499,"Professional, Scientific, and Technical Services",Mission,POINT (-122.419 37.757675),-122.419,37.757675
6,Bakesale Betty/hot Dog Bill East 600,1 Warriors Way Lvl 600 East,San Francisco,CA,94158,07/01/1987,NaN,05/28/2019,NaN,7220-7229,Food Services,Mission Bay,POINT (-122.386406 37.768967),-122.386406,37.768967


In [17]:
merged_df['latitude'].isna().sum()

0

In [18]:
#change date format
# merged_df = (merged_df[ merged_df.iloc[:,5:8 ]]).apply(pd.to_datetime)

# merged_df['busi_start_dt']= pd.to_datetime(merged_df['busi_start_dt'])
# merged_df['busi_end_dt']= pd.to_datetime(merged_df['busi_end_dt'])
# merged_df['loc_start_dt']= pd.to_datetime(merged_df['loc_start_dt'])
# merged_df['loc_end_dt']= pd.to_datetime(merged_df['loc_end_dt'])
# merged_df

In [19]:
# #extract year from date format
# merged_df['busi_start_yr'] = pd.DatetimeIndex(merged_df['busi_start_dt']).year
# merged_df['busi_end_yr'] = pd.DatetimeIndex(merged_df['busi_end_dt']).year
# merged_df['loc_start_yr'] = pd.DatetimeIndex(merged_df['loc_start_dt']).year
# merged_df['loc_end_yr'] = pd.DatetimeIndex(merged_df['loc_end_dt']).year
# merged_df

In [20]:
# count2018 = merged_df[pd.DatetimeIndex(merged_df['busi_start_dt']).year >2009]
# count2018['busi_start_dt'].count()

In [21]:
# merged_df = merged_df.loc[ merged_df['busi_start_yr'] > 2009].reset_index()
# merged_df

In [22]:
# merged_df.iloc[2]

In [23]:
# merged_df = merged_df[ merged_df.iloc[:,5:8 ]].astype(str)
# merged_df.head()

In [24]:
# merged_df['busi_start_dt']= (merged_df['busi_start_dt']).replace('NaT', None)
# merged_df['busi_end_dt']= (merged_df['busi_end_dt']).replace('NaT', None)
# merged_df['loc_start_dt']= (merged_df['loc_start_dt']).replace('NaT', None)
# merged_df['loc_end_dt']= (merged_df['loc_end_dt']).replace('NaT', None)
# merged_df.head()
# merged_df = merged_df[ merged_df.iloc[:,5:8 ]].replace('NaT', None)
# merged_df.head()

In [25]:
# merged_df.replace({np.NaN: None})
# merged_df.head()

In [26]:
# merged_df['busi_start_dt']=(merged_df['busi_start_dt']).astype(str)
# merged_df['busi_end_dt']=(merged_df['busi_end_dt']).astype(str)
# merged_df['loc_start_dt']= (merged_df['loc_start_dt']).astype(str)
# merged_df['loc_end_dt']= (merged_df['loc_end_dt']).astype(str)
# merged_df.head()

In [27]:
merged_df.dropna(subset = ['NAICS_code', 'busi_type','neighborhood'], how='any', inplace=True)
merged_df.head()

In [28]:
# multiBusiness = merged_df[ merged_df['business_type'] == "Multiple" ].index

# drop_multi_df= merged_df.drop(multiBusiness, inplace=True)
# merged_df.head()

# Loading database

In [29]:
# extract the DataFrame to an output csv file 
# merged_df.to_csv('sf_data.csv', encoding='utf-8', index=False)

In [30]:
# sfdf=pd.read_csv("sf_data.csv")
# sfdf.head()

In [31]:
# dropdf = pd.read_csv('drop_dates.csv')
# dropdf.head()

In [32]:
#Create database
engine = create_engine('postgresql://postgres:'+ pswd + '@localhost:5432/sfbusiness_db')
if not database_exists(engine.url):
    create_database(engine.url)

print(database_exists(engine.url))

In [33]:
#Establish connection with the database
connection = engine.connect()

In [34]:
#Load the data to SQL database (a table named 'dow_jones' is created in postgresql)
merged_df.to_sql(name='sf_business', con=engine, if_exists='replace', index=False)

In [35]:
#Query the SQL database
# pd.read_sql_query('select * from sf_business limit 10', con=engine).head()

# Extra work

In [36]:
#drop null values from busi/loc end dates
# drop_dates_df = merged_df.dropna(subset =['busi_end_dt', 'loc_end_dt'], how='all').reset_index(drop=True)
# drop_dates_df.head()

In [37]:
# drop_dates_df['busi_end_dt'].isna().sum()

In [38]:
# extract the DataFrame with dropped null values to an output csv file 
# drop_dates_df.to_csv('drop_dates.csv', encoding='utf-8', index=False)

In [39]:
#Establish connection with the database
# engine = create_engine('postgresql://postgres:'+ pswd + '@localhost:5432/sf_drop_db')
# connection = engine.connect()

In [40]:
#Load the data to SQL database (a table named 'dow_jones' is created in postgresql)
# drop_dates_df.to_sql(name='drop_data', con=engine, if_exists='replace', index=False)

In [41]:
#Query the SQL database
# pd.read_sql_query('select * from drop_data', con=engine).head()